In [1]:
dados ='/home/dev/PycharmProjects/FallDetection/conv/Novo cj de dados/Conv1d/Dominio da Frequencia/Todos os arrays/dados_freq.npy'
rotulos ='/home/dev/PycharmProjects/FallDetection/conv/Novo cj de dados/rotulos.npy'

In [2]:
import numpy as np
X = np.load(dados)
y = np.load(rotulos)

In [3]:
from sklearn.model_selection import train_test_split

# Dividindo os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dividindo o conjunto de treinamento em conjunto de treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



In [4]:
X_train.shape

(1365, 200, 1)

In [5]:
X_test.shape

(427, 200, 1)

In [6]:
X_val.shape

(342, 200, 1)

In [7]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

2023-06-27 15:16:26.346497: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-27 15:16:27.930039: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-27 15:16:27.936247: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-27 15:16:32.746697: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D
import optuna

def objective(trial):
    # Definindo o espaço de busca dos hiperparâmetros
    filter_size = trial.suggest_int('filter_size', 8, 256, log=True)
    kernel_size = trial.suggest_int('kernel_size', 2, 7)
    num_layers = trial.suggest_int('num_layers', 2, 6)
    dense_neurons = trial.suggest_int('dense_neurons', 60, 320, log=True)
    max_pool = 2
    dropout = trial.suggest_int('dropout', 2, 5, step=1) / 10.0
    learning_rate = trial.suggest_categorical('learning_rate', [0.0001, 0.0003,0.0006,0.001,0.003,0.006, 0.01])

    model = keras.Sequential()

    for i in range(num_layers):
        if i == 0:
            model.add(Conv1D(filters=filter_size, kernel_size=kernel_size, activation="relu", input_shape=(200, 1)))
            model.add(MaxPooling1D(pool_size=max_pool))
            model.add(Dropout(dropout))
        else:
            if filter_size < kernel_size:
                filter_size = kernel_size
            filter_size *= 2
            model.add(Conv1D(filters=filter_size, kernel_size=kernel_size, activation="relu"))
            model.add(MaxPooling1D(pool_size=max_pool))
            model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(dense_neurons, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    historico = model.fit(X_train, y_train, batch_size=32, epochs=35, validation_data=(X_val, y_val), verbose=1)

    return historico.history['val_accuracy'][-1]




In [9]:
import csv

# criação do objeto de estudo
study = optuna.create_study(direction="maximize")

# abertura do arquivo CSV em modo de escrita
with open("results.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(["trial", "accuracy", "hyperparameters"])  # cabeçalho

    # iteração dos trials
    
    study.optimize(objective, n_trials=10)
    best_trial = study.best_trial
    print("Trial {}: Accuracy = {}".format(i, best_trial.value))

    # escrita dos valores no arquivo CSV
    writer.writerow([i, best_trial.value, best_trial.params])

    print("Trial {}: Accuracy = {}".format(i, best_trial.value))

print(study.best_params)

[I 2023-06-27 15:16:39,019] A new study created in memory with name: no-name-5bf49505-18a5-4bb3-a3c0-4208cd7ae058
2023-06-27 15:16:40.566347: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-27 15:16:40.592613: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/35
43/43 [==============================] - 3s 26ms/step - loss: 0.8955 - accuracy: 0.9055 - val_loss: 0.1251 - val_accuracy: 0.9503
Epoch 2/35
43/43 [==============================] - 1s 21ms/step - loss: 0.0697 - accuracy: 0.9758 - val_loss: 0.0999 - val_accuracy: 0.9737
Epoch 3/35
43/43 [==============================] - 1s 21ms/step - loss: 0.0403 - accuracy: 0.9861 - val_loss: 0.0199 - val_accuracy: 0.9942
Epoch 4/35
43/43 [==============================] - 1s 21ms/step - loss: 0.0508 - accuracy: 0.9802 - val_loss: 0.0380 - val_accuracy: 0.9883
Epoch 5/35
43/43 [==============================] - 1s 21ms/step - loss: 0.0355 - accuracy: 0.9905 - val_loss: 0.0409 - val_accuracy: 0.9942
Epoch 6/35
43/43 [==============================] - 1s 21ms/step - loss: 0.0314 - accuracy: 0.9905 - val_loss: 0.0096 - val_accuracy: 0.9971
Epoch 7/35
43/43 [==============================] - 1s 21ms/step - loss: 0.0334 - accuracy: 0.9890 - val_loss: 0.0240 - val_accuracy: 0.9883
Epoch 8/35
43

[I 2023-06-27 15:17:14,748] Trial 0 finished with value: 0.9736841917037964 and parameters: {'filter_size': 51, 'kernel_size': 2, 'num_layers': 3, 'dense_neurons': 240, 'dropout': 3, 'learning_rate': 0.01}. Best is trial 0 with value: 0.9736841917037964.


Epoch 1/35
43/43 [==============================] - 3s 66ms/step - loss: 1.2912 - accuracy: 0.9092 - val_loss: 0.0359 - val_accuracy: 0.9825
Epoch 2/35
43/43 [==============================] - 3s 62ms/step - loss: 0.0350 - accuracy: 0.9846 - val_loss: 0.0083 - val_accuracy: 0.9971
Epoch 3/35
43/43 [==============================] - 3s 62ms/step - loss: 0.0473 - accuracy: 0.9846 - val_loss: 0.0299 - val_accuracy: 0.9942
Epoch 4/35
43/43 [==============================] - 3s 63ms/step - loss: 0.0772 - accuracy: 0.9751 - val_loss: 0.1143 - val_accuracy: 0.9737
Epoch 5/35
43/43 [==============================] - 3s 63ms/step - loss: 0.0579 - accuracy: 0.9758 - val_loss: 0.2058 - val_accuracy: 0.9415
Epoch 6/35
43/43 [==============================] - 3s 64ms/step - loss: 0.0548 - accuracy: 0.9773 - val_loss: 0.0794 - val_accuracy: 0.9649
Epoch 7/35
43/43 [==============================] - 3s 63ms/step - loss: 0.0387 - accuracy: 0.9824 - val_loss: 0.0186 - val_accuracy: 0.9912
Epoch 8/35
43

[I 2023-06-27 15:18:50,447] Trial 1 finished with value: 0.9912280440330505 and parameters: {'filter_size': 161, 'kernel_size': 7, 'num_layers': 2, 'dense_neurons': 194, 'dropout': 3, 'learning_rate': 0.006}. Best is trial 1 with value: 0.9912280440330505.


...

Epoch 1/35
43/43 [==============================] - 1s 10ms/step - loss: 0.8702 - accuracy: 0.8652 - val_loss: 0.0494 - val_accuracy: 0.9971
Epoch 2/35
43/43 [==============================] - 0s 7ms/step - loss: 0.1082 - accuracy: 0.9553 - val_loss: 0.0222 - val_accuracy: 0.9971
Epoch 3/35
43/43 [==============================] - 0s 8ms/step - loss: 0.0592 - accuracy: 0.9766 - val_loss: 0.0218 - val_accuracy: 0.9942
Epoch 4/35
43/43 [==============================] - 0s 7ms/step - loss: 0.0602 - accuracy: 0.9758 - val_loss: 0.0328 - val_accuracy: 0.9971
Epoch 5/35
43/43 [==============================] - 0s 7ms/step - loss: 0.0585 - accuracy: 0.9788 - val_loss: 0.0309 - val_accuracy: 0.9971
Epoch 6/35
43/43 [==============================] - 0s 7ms/step - loss: 0.0388 - accuracy: 0.9846 - val_loss: 0.0525 - val_accuracy: 0.9854
Epoch 7/35
43/43 [==============================] - 0s 7ms/step - loss: 0.0630 - accuracy: 0.9758 - val_loss: 0.0201 - val_accuracy: 0.9912
Epoch 8/35
43/43 [=

[I 2023-06-27 15:21:59,252] Trial 8 finished with value: 0.9824561476707458 and parameters: {'filter_size': 16, 'kernel_size': 5, 'num_layers': 2, 'dense_neurons': 246, 'dropout': 4, 'learning_rate': 0.01}. Best is trial 6 with value: 1.0.


Epoch 1/35
43/43 [==============================] - 1s 11ms/step - loss: 0.3288 - accuracy: 0.8630 - val_loss: 0.0825 - val_accuracy: 0.9942
Epoch 2/35
43/43 [==============================] - 0s 8ms/step - loss: 0.0756 - accuracy: 0.9766 - val_loss: 0.0139 - val_accuracy: 0.9971
Epoch 3/35
43/43 [==============================] - 0s 8ms/step - loss: 0.0482 - accuracy: 0.9846 - val_loss: 0.0114 - val_accuracy: 1.0000
Epoch 4/35
43/43 [==============================] - 0s 8ms/step - loss: 0.0470 - accuracy: 0.9839 - val_loss: 0.0286 - val_accuracy: 0.9942
Epoch 5/35
43/43 [==============================] - 0s 8ms/step - loss: 0.0361 - accuracy: 0.9868 - val_loss: 0.0090 - val_accuracy: 1.0000
Epoch 6/35
43/43 [==============================] - 0s 8ms/step - loss: 0.0237 - accuracy: 0.9912 - val_loss: 0.0265 - val_accuracy: 0.9942
Epoch 7/35
43/43 [==============================] - 0s 8ms/step - loss: 0.0311 - accuracy: 0.9905 - val_loss: 0.0282 - val_accuracy: 0.9854
Epoch 8/35
43/43 [=

[I 2023-06-27 15:22:11,935] Trial 9 finished with value: 0.9795321822166443 and parameters: {'filter_size': 11, 'kernel_size': 4, 'num_layers': 4, 'dense_neurons': 151, 'dropout': 2, 'learning_rate': 0.003}. Best is trial 6 with value: 1.0.


NameError: name 'i' is not defined